In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [ ]:
TRN_cat = pd.HDFStore('/home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA/TRN_navi_catalog.h5')
TRN_cat_df = TRN_cat['df']
TRN_cat_df.head()

In [ ]:
def verify(save, tol = 3):
    c1_t1 = [save[0].x1, save[0].y1, save[0].r1]
    c2_t1 = [save[0].x2, save[0].y2, save[0].r3]
    c3_t1 = [save[0].x3, save[0].y2, save[0].r3]

    c1_t2 = [save[1].x1, save[1].y1, save[1].r1]
    c2_t2 = [save[1].x2, save[1].y2, save[1].r3]
    c3_t2 = [save[1].x3, save[1].y2, save[1].r3]

    centroid_1 = [(c1_t1[0]+c2_t1[0]+c3_t1[0])/3, (c1_t1[1]+c2_t1[1]+c3_t1[1])/3]
    centroid_2 = [(c1_t2[0]+c2_t2[0]+c3_t2[0])/3, (c1_t2[1]+c2_t2[1]+c3_t2[1])/3]

    delta_x = abs(centroid_1[0]-centroid_2[0])
    delta_y = abs(centroid_1[1]-centroid_2[1])

    if c1_t1[2]-c1_t2[2] < tol: # Se il primo cratere è il primo cratere bis
        if (abs(abs(c1_t1[0]-c1_t2[0])-delta_x) < tol) & (abs(abs(c1_t1[1]-c1_t2[1])-delta_y) < tol):
            return True
    elif c1_t1[2]-c2_t2[2] < tol: # primo - secondo
        if (abs(abs(c1_t1[0]-c2_t2[0])-delta_x) < tol) & (abs(abs(c1_t1[1]-c2_t2[1])-delta_y) < tol):
            return True
    elif c1_t1[2]-c2_t2[2] < tol: # primo - terzo
        if (abs(abs(c1_t1[0]-c3_t2[0])-delta_x) < tol) & (abs(abs(c1_t1[1]-c3_t2[1])-delta_y) < tol):
            return True
    else: return False

In [ ]:
############################################
dt = 10
############################################
df = pd.read_csv(f'DATA/ephemeris sat/inclination zero/{dt} step size.csv', header=3, sep=';') 
real_Latitudes, real_Longitudes, real_Altitudes = df['Lat (deg)'], df['Lon (deg)'], df['Alt (km)']
real_Vxs,real_Vys,real_Vzs = df['x (km/sec)'], df['y (km/sec)'],df['z (km/sec)']

real_X, real_Y, real_Z = [], [], []
for i in range(len(df)):
    altitude = real_Altitudes[i]
    latitude = real_Latitudes[i]
    longitude = real_Longitudes[i]
    x, y, z = spherical2cartesian(altitude, latitude, longitude)
    real_X.append(x)
    real_Y.append(y)
    real_Z.append(z)
real_X, real_Y, real_Z = np.array(real_X),np.array(real_Y),np.array(real_Z)
x, y, z = real_X[0], real_Y[0], real_Z[0]


# LOAD ALL IMAGES:
dt = 10
dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt

In [ ]:
idx = 0
tol = 5

# IMG1:
img1=cv2.imread(dict[str(idx+1)])
craters_det1 = detect(img1)
craters_cat1 = crater_catalogued([x,y,z]) # OUTPUTS: x,y, radius (px)


# Pandas DataFrame:
df_craters_det1 = sort_mat(craters_det1)
# Find all triplets:
H1 = find_all_triplets(craters_det1)
H2 = find_all_triplets(craters_cat1)
QUERY1= pd.DataFrame(H1, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
QUERY2= pd.DataFrame(H2, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
# remove wrong triplet:
QUERY1 =QUERY1[ (QUERY1.Angle1 > 10) & (QUERY1.Angle2 > 10) & (QUERY1.Angle3 > 10) ].reset_index(drop=True)
QUERY2 =QUERY2[ (QUERY2.Angle1 > 10) & (QUERY2.Angle2 > 10) & (QUERY2.Angle3 > 10) ].reset_index(drop=True)
##########################################
# SEARCH
##########################################
found = False
for idx in range(QUERY1.shape[0]):
    Angle1_Q1 = QUERY1.Angle1[idx]
    Angle2_Q1 = QUERY1.Angle2[idx]
    Angle3_Q1 = QUERY1.Angle3[idx]
    d1_Q1 = QUERY1.des1[idx]
    d2_Q1 = QUERY1.des2[idx]
    d3_Q1 = QUERY1.des3[idx]

    SEARCH = QUERY2[ ((abs(QUERY2.Angle1 - Angle1_Q1) < tol) & (abs(QUERY2.Angle2 - Angle2_Q1) < tol) & (abs(QUERY2.Angle3 - Angle3_Q1) < tol))   \
                      |  ((abs(QUERY2.Angle1 - Angle2_Q1) < tol) & (abs(QUERY2.Angle2 - Angle3_Q1) < tol) & (abs(QUERY2.Angle3 - Angle1_Q1) < tol)) \
                      |  ((abs(QUERY2.Angle1 - Angle3_Q1) < tol) & (abs(QUERY2.Angle2 - Angle1_Q1) < tol) & (abs(QUERY2.Angle3 - Angle2_Q1) < tol))]

    SEARCH = SEARCH  [ ((abs(QUERY2.des1- d1_Q1) < tol) & (abs(QUERY2.des2- d2_Q1) < tol) & (abs(QUERY2.des3- d3_Q1) < tol)) \
                     | ((abs(QUERY2.des1- d2_Q1) < tol) & (abs(QUERY2.des2- d3_Q1) < tol) & (abs(QUERY2.des3- d1_Q1) < tol)) \
                     | ((abs(QUERY2.des1- d3_Q1) < tol) & (abs(QUERY2.des2- d1_Q1) < tol) & (abs(QUERY2.des3- d2_Q1) < tol)) ]
    if SEARCH.shape[0] != 0:
        print('Maybe I found!')
        save = [QUERY1.iloc[idx], SEARCH.iloc[0]]
        switch = verify(save, tol = 10)
        if switch:
            print('FOUND!')
            found = True
            break
        else:
            continue
